## basic config

In [1]:
import torch
torch.cuda.device_count()

1

In [21]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [3]:
# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/home/bean/"

# keep matching encoder,SG as pairt 
old = {"encoder":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
       "SG": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
       "model_name":"stylegan2",
       "resolution":1024,
       }

new = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl",
         "model_name":"stylegan3",
               "resolution":1024,
         }

new_256 = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/Stylegan3_1_files/stylegan3-r-ffhqu-256x256.pkl",
         "model_name":"stylegan3",
         "resolution":256,
         }

videos = {
       "obama": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/obama.mp4",
       "man_speaking": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
       "rahul": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/rahul.mp4",
       "alien_girl": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/alien_girl.mp4",
       "mirrAR": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/mirrAR.mp4",
       "vsauce_frnd":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4",
       
}

In [4]:
EXPERIMENT_NAME_POSTFIX = "vsauce_ffmpeg"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"
models_set = new
Testing =True
# --------------------------------  


import datetime
lazy_config = {
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+"_"+ EXPERIMENT_NAME_POSTFIX,
    "e4e": models_set['encoder'],
    "stylegan2_ada_ffhq": models_set['SG'],
    "video_path":  videos["vsauce_frnd"],
    "model_name" : models_set['model_name'],
    "n_frames" : 1,#120,#200,
    "IMAGE_SIZE": models_set["resolution"],
    "N_STEPS_in_draggan":  "150",
    "editing_function_name":"large_eyes",#"large_eyes" # "make_jaw_wider" # "mouth_wide"
    "fps": 24,
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI

hyper_config = {
    "max_pti_steps": 350,
    "first_inv_steps":450,
    "max_images_to_invert": 200,
}

if Testing: 
    hyper_config["max_pti_steps"] = 1
    hyper_config["first_inv_steps"] = 1
    hyper_config["max_images_to_invert"] = 5
    lazy_config["N_STEPS_in_draggan"] = '1'
    


# ----------------------------------------------------------------------------------------------

from run_utils import *

# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])



# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
     "quad_values_path": os.path.join(Experiment_path,'quad_values'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [ ]:
# extract frames from video
#==============================================================================

from importlib.machinery import SourceFileLoader
# imports the module from the given path
# video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()

ffmpeg_utils = SourceFileLoader("video_utils","../utils_draggan/ffmpeg_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")

# video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])
ffmpeg_utils.ffmpeg.extract_frames(video_path = lazy_config['video_path'],
                                      output_frames_dir = raw_path,
                                      fps = lazy_config['fps'],
                                      first_n_frames = lazy_config['n_frames'],
)

# start

In [7]:
# # # delete this cell 
# # "/home/bean/DragVideo/Data_store/man_speaking/aligned": input_data_path 
# # "/home/bean/DragVideo/Data_store/man_speaking/quad_values", quad_pkl_path

# raw_path = "/home/bean/DragVideo/Data_store/man_speaking/raw"
# pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path ,quad pickle dir


In [8]:
from utils.align_data import pre_process_images
pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path ,quad pickle dir
#
# align_faces_2(os.path.join(Experiment_path,'raw'))



100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

saving aligned images...
saving quad values...
done!


In [ ]:
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

generator_type =paths_config.multi_id_model_type if use_multi_id_training else "__"
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

# print(sg_tuned_pkl) # 'QBUXQCXZGWET'



#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)
print("pwd",os.getcwd())
import sys
sys.path.append(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)


# Runs draggan 
all the the below cells use only these config values

In [22]:
if False:
    # just to run draggan seperately
    import os 
    
    a = "/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_untuned_sg"
    b = "/home/bean/DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl"

    Experiment_path = a# f"/home/bean/DragVideo/Data_store/experiments/{lazy_config['EXP_NAME']}"	
    sg_tuned_pkl = b #new["SG"]


    lazy_config["N_STEPS_in_draggan"]="1"
    lazy_config["editing_function_name"]="large_eyes"
    lazy_config["IMAGE_SIZE"]="1024"


In [ ]:
# clean all data from gpu
import torch
torch.cuda.empty_cache()

os.chdir(env_config["DragGAN_dir"])
import subprocess

print(f"sg_tuned_pkl: {sg_tuned_pkl}")

subprocess.call(['python', '_run_dragvideo.py',
                 '--Experiment_path', Experiment_path,
                 '--N_STEPS',       lazy_config["N_STEPS_in_draggan"],
                 '--CHECKPOINT_PATH', sg_tuned_pkl,
                 "--MAX_SIZE",      str(lazy_config["IMAGE_SIZE"]),
                 "--editing_function_name",lazy_config["editing_function_name"],
])
                #  "--verbose",       "False"])


# post processing

In [24]:
import os
from tqdm import tqdm
os.getcwd()
# import importlib
# importlib.reload(ffmpeg_utils)

'/home/bean/DragVideo/DragGAN'

In [25]:

from utils import de_alignment 
put_back_the_edited_image = de_alignment.put_back_the_edited_image


In [26]:
def paste_edited_faces_back(dir_path,edited_dir='after_drag',save_dir='after_drag_pasted'):
    raw_dir = os.path.join(dir_path,'raw')
    edited_dir = os.path.join(dir_path,edited_dir)
    quad_dir = os.path.join(dir_path,'quad_values')
    save_dir = os.path.join(dir_path,save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # get names from edited_dir
    names = [name.split('.')[0] for name in os.listdir(edited_dir)]
    for name in tqdm(names):
        raw_image = os.path.join(raw_dir,name+'.png')
        edited_image = os.path.join(edited_dir,name+'.png')
        quad_path = os.path.join(quad_dir,name+'.pkl')
        save_path = os.path.join(save_dir,name+'.png')
        put_back_the_edited_image(raw_image,edited_image,quad_path,save_path=save_path)


In [ ]:
paste_edited_faces_back(Experiment_path,edited_dir="before_drag",save_dir="before_drag_pasted")
paste_edited_faces_back(Experiment_path,edited_dir="after_drag",save_dir="after_drag_pasted")

In [32]:
# =============================================================================
# using ffmpeg to make video
# =============================================================================
before_drag_pasted_dir = os.path.join(Experiment_path,'before_drag_pasted')
after_drag_pasted_dir = os.path.join(Experiment_path,'after_drag_pasted')
videos_dir = os.path.join(Experiment_path,'videos')

ffmpeg_utils.ffmpeg.make_video(before_drag_pasted_dir,
                               video_name="pre_drag_full",
                               video_dir=videos_dir,
                               fps=lazy_config['fps'],
                                 )
ffmpeg_utils.ffmpeg.make_video(after_drag_pasted_dir,
                                 video_name="post_drag_full",
                                    video_dir=videos_dir,
                                    fps=lazy_config['fps'],
                                    )


ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

In [33]:
# =============================================================================
# hstack videos
# =============================================================================

video1= os.path.join(videos_dir,"pre_drag_full.mp4")
video2= os.path.join(videos_dir,"post_drag_full.mp4")

# pre_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(video1,
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_pre_post_drag_full",
)

# original_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(lazy_config['video_path'],
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_original_post_drag_full",
)

/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_untuned_sg/videos/hstack_pre_post_drag_full.mp4


ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_untuned_sg/videos/hstack_original_post_drag_full.mp4


Output #0, mp4, to '/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_untuned_sg/videos/hstack_original_post_drag_full.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.12.100
    Stream #0:0: Video: h264 (libx264) (avc1 / 0x31637661), yuv420p, 2560x720 [SAR 1:1 DAR 32:9], q=-1--1, 25 fps, 12800 tbn, 25 tbc (default)
    Metadata:
      encoder         : Lavc58.18.100 libx264
    Side data:
      cpb: bitrate max/min/avg: 0/0/0 buffer size: 0 vbv_delay: -1
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      encoder         : Lavc58.18.100 aac
Past duration 0.958977 too large
Past duration 0.999992 too large
frame=  127 fps= 81 q=-1.0 Lsize=    1294kB time=00:00:05.45 bitrate=1942.8kbits/s dup=0 drop=112 speed= 3.5x    
video:1203kB audio:85kB subtitle:0kB other streams:0kB globa